In [1]:
import os ## importing all the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import time

In [2]:
modules = os.listdir(os.getcwd() + "/Modules")
modules 

['PGGen.v',
 'top_CLA8.v',
 'top_CLA16.v',
 'top_CLA32.v',
 'top_FIFO.sv',
 'counter.sv',
 'frequency_div.sv',
 'top_traffic_controller.sv',
 'traffic_controller.sv',
 'top_BA_4x4_Multiplier.v',
 'top_mealy.v',
 'top_moore.v',
 'top_priority_encoder0.v',
 'top_priority_encoder1.v',
 'top_RCA.v',
 'clkdiv.v',
 'full_adder.v',
 'half_adder.v',
 'top_convolution.sv',
 'top_W_2R1_Reg_file.sv']

## Adding the .v files and .sv files

In [10]:
arguments = ""
for module in modules:
    arguments += "Modules/" + module + " "

# print(arguments) ## locations of the files

In [11]:
os.system("vivado -mode batch -source tcl1_add.tcl -tclargs {}".format(arguments))

0

## Running the synthesis and implementation

In [4]:
top_modules = [module for module in modules if "top" in module] 
# top_modules

In [5]:
start = time.time()

if os.path.exists("Impl"):
    res = input("Do you want to delete existing directory '{}' to replace new files? (y/n)".format("Impl")) #asking the user to delete the existing Impl result folder 
    if res.lower() == "y": 
        os.system("rm -r {}".format("Impl")) # if response is yes then delete Impl folder
        os.mkdir("Impl") # making new directory named as Impl
else:
    os.mkdir("Impl") # if there is no existing Impl then you can simply create a new folder 
        
for file in top_modules: # running the implementation and syntesis for every top_module
    if ".sv" in file:
        location = "Impl/" + file[:-3] # creating the folders for every .sv files named with removed suffix
    else:
        location = "Impl/" + file[:-2] # creating the folders for every .v files named with removed suffix
        
    if os.path.exists(location):
        response = input("Do you want to delete existing directory '{}' to replace with new files? (y/n)".format(location))
        if response.lower() == "y": ## if y then create a new folder and run the files again
            os.system("rm -r {}".format(location)) ## clearing the previous directory 
            os.mkdir(location) ## making a new diectory
            os.system("vivado -mode batch -source tcl1_run.tcl -tclargs {}".format(file)) ## running the new files
        else:
            continue
    else:
        os.mkdir(location) ## if no location is present then create new location
        os.system("vivado -mode batch -source tcl1_run.tcl -tclargs {}".format(file)) ## running the files
    
end = time.time()
time_taken = end-start # tracking the time
print("Total time taken to implement the files: ", time_taken, end = 's')


Do you want to delete existing directory 'Impl' to replace new files? (y/n) n
Do you want to delete existing directory 'Impl/top_CLA8' to replace with new files? (y/n) n
Do you want to delete existing directory 'Impl/top_CLA16' to replace with new files? (y/n) n
Do you want to delete existing directory 'Impl/top_CLA32' to replace with new files? (y/n) n
Do you want to delete existing directory 'Impl/top_FIFO' to replace with new files? (y/n) n
Do you want to delete existing directory 'Impl/top_traffic_controller' to replace with new files? (y/n) n
Do you want to delete existing directory 'Impl/top_BA_4x4_Multiplier' to replace with new files? (y/n) n
Do you want to delete existing directory 'Impl/top_mealy' to replace with new files? (y/n) n
Do you want to delete existing directory 'Impl/top_moore' to replace with new files? (y/n) n
Do you want to delete existing directory 'Impl/top_priority_encoder0' to replace with new files? (y/n) n
Do you want to delete existing directory 'Impl/top

Total time taken to implement the files:  1326.8133342266083s

## Formatting the data and storing the data we want into .csv file

In [7]:
module_names = os.listdir("Impl") # seeing what folders are all present in the Impl result folder
No_of_modules = len(module_names)
# intializing all of the lists which will be needed 
signal_power_list = [] 
logic_power_list = [] 
sp_list = [] 
dc_power_list = [] 
tonp_list = []
dpd_list = []
F7_Mux_list = []
LUT_Mem_list = []
LUT_logic_list = []
Slice_LUT_list = []
LUT1_list = []
LUT2_list = []
LUT3_list = []
LUT4_list = []
LUT5_list = []
LUT6_list = []
LUT7_list = []
LUTs_list = np.zeros((7, No_of_modules), dtype = int)
LUTs_list = [LUT1_list, LUT2_list, LUT3_list, LUT4_list, LUT5_list, LUT6_list, LUT7_list] # storing all the LUTs data
parameters_list = [signal_power_list, logic_power_list, tonp_list, dc_power_list, sp_list, dpd_list, F7_Mux_list, LUT_Mem_list, LUT_logic_list, Slice_LUT_list,LUT1_list, LUT2_list, LUT3_list, LUT4_list, LUT5_list, LUT6_list, LUT7_list] 
counter = 0 # this counter will be used to check all of the parameters are loaded correctly as per of no_of_modules
counter_1 = 0 # there would be some repetitions of "LUT as Memory" and "LUT as logic" to keep track on them such that we will not load them two times. We will keep these counters.
counter_2 = 0
# print(module_names)

# we will be taking all of the required parameters from respective txt files 
for MODULE in module_names:
    counter = counter + 1
    with open("Impl/{}/power.txt".format(MODULE)) as file: # storing the txt file data as file 
        lines = file.readlines() # spltting them into lines
    # Getting the required values by searching the prefix text and getting the required values into the lists
    for line in lines:
        if "Total On-Chip Power" in line:
            tonp_line = line.split()
            if "Junction temp exceeded" in line:
                tonp = float(tonp_line[-5])
            else:
                tonp = float(tonp_line[-2])
            tonp_list.append(tonp)
        if "Static Power" in line:
            sp_line = line.split()
            sp = float(sp_line[-5])
            sp_list.append(sp)
        if "Signals" in line:
            signal_power_line = line.split()
            signal_power = float(signal_power_line[3])
            signal_power_list.append(signal_power) 
        if "Slice Logic" in line:
            logic_power_line = line.split()
            logic_power = float(logic_power_line[4])
            logic_power_list.append(logic_power) 
                
    with open("Impl/{}/timing.txt".format(MODULE)) as file:
        lines = file.readlines()
    for line in lines:
        if "Static Power" in line:
            sp_line = line.split()
            sp = float(sp_line[-5])
            sp_list.append(sp)
        if "Data Path Delay" in line:
            dpd_line = line.split()
            dpd = float(dpd_line[3][:-2]) # nano seconds(ns)
            dpd_list.append(dpd)
                
    with open("Impl/{}/utilization.txt".format(MODULE)) as file:
        lines = file.readlines()
    for line in lines:
        if "F7 Muxes" in line:
            F7_Mux_line = line.split()
            F7_Mux = int(F7_Mux_line[4]) 
            F7_Mux_list.append(F7_Mux)
        if "LUT as Memory" in line:
            LUT_Mem_line = line.split()
            if(counter_1 == 0):
                LUT_Mem_list.append(int(LUT_Mem_line[5]))
                counter_1 = counter_1 + 1
            else:
                counter_1 = 0
        if "LUT as Logic" in line:
            LUT_logic_line = line.split()
            if(counter_1 == 0):
                LUT_logic_list.append(int(LUT_logic_line[5]))
                counter_2 = counter_2 + 1
            else:
                counter_2 = 0
        if "Slice LUTs" in line:
            Slice_LUT_line = line.split()
            Slice_LUT = int(Slice_LUT_line[4])
            Slice_LUT_list.append(Slice_LUT)
        if "LUT1" in line:
            LUT1_line = line.split()
            LUT1 = int(LUT1_line[3])
            LUT1_list.append(LUT1)
        if "LUT2" in line:
            LUT2_line = line.split()
            LUT2 = int(LUT2_line[3])
            LUT2_list.append(LUT2)
        if "LUT3" in line:
            LUT3_line = line.split()
            LUT3 = int(LUT3_line[3])
            LUT3_list.append(LUT3)
        if "LUT4" in line:
            LUT4_line = line.split()
            LUT4 = int(LUT4_line[3])
            LUT4_list.append(LUT4)
        if "LUT5" in line:
            LUT5_line = line.split()
            LUT5 = int(LUT5_line[3])
            LUT5_list.append(LUT5)
        if "LUT6" in line:
            LUT6_line = line.split()
            LUT6 = int(LUT6_line[3])
            LUT6_list.append(LUT6) 
        if "LUT7" in line:
            LUT7_line = line.split()
            LUT7 = int(LUT7_line[3])
            LUT7_list.append(LUT7)
                
    for i in range(len(parameters_list)):
        if(len(parameters_list[i]) != counter): # make sure all of the arrays are maintained at same size by adding zeros
            parameters_list[i].append(0)

Slice_LUT_list_1 = np.zeros((No_of_modules),dtype = int) # the total LUTs present from LUT1 to LUT7
LUTs_difference = np.zeros((No_of_modules),dtype = int) # calculating the difference between Slice_LUTs2 and Slice_LUTs
dc_power_list = np.zeros((No_of_modules)) # addition of signal and logic powers
LUT_logic_list_1 = np.zeros((No_of_modules),dtype = int) 

for i in range(No_of_modules):
    for j in range(7):
        Slice_LUT_list_1[i] += LUTs_list[j][i] # adding the 7LUTs and storing them in Slice_LUT_list_1 for every module


LUT_logic_list_1 = Slice_LUT_list_1 

for i in range(No_of_modules):
    LUTs_difference[i] = Slice_LUT_list_1[i] - Slice_LUT_list[i] # calculating the difference between Slice_LUTs2 and Slice_LUTs
    
for i in range(No_of_modules):
    dc_power_list[i] = signal_power_list[i] + logic_power_list[i] # addition of signal and logic powers

In [8]:
data = [module_names, signal_power_list, logic_power_list, sp_list, dc_power_list, tonp_list, dpd_list, F7_Mux_list, LUT_Mem_list, LUT_logic_list, Slice_LUT_list, LUT1_list, LUT2_list, LUT3_list, LUT4_list, LUT5_list, LUT6_list, LUT7_list, Slice_LUT_list_1, LUT_logic_list_1, LUTs_difference]
heading = ["Module_names", "Signal_power", "Logic_power","Static_Power(W)", "DC_power(W)", "Total_on_chip_power(W)", "Data_Path_Delay(ns)", "F7_Muxes", "LUTs_as_Memory", "LUTs_as_Logic", "Slice_LUTs", "LUT1", "LUT2", "LUT3", "LUT4", "LUT5", "LUT6", "LUT7", "Slice_LUTs2", "LUTs_as_logic2", "LUTs_difference"]

dic = {}
for i in range(len(heading)): ## storing all the values into dictonary 
    dic[heading[i]] = data[i]

df = pd.DataFrame(dic) # storing the data in dataframe

if(os.path.exists("results.csv")):
    response = input("Do you want to replace '{}' this csv file?(y/n)".format("results.csv")) # taking the user response to delete the existing resultss.csv file or not.
    if(response.lower() == "y"):
        os.system("rm -r {}".format("results.csv"))
        df.to_csv("results.csv", index = None)
else:
    df.to_csv("results.csv", index = None)

Do you want to replace 'results.csv' this csv file?(y/n) y


In [9]:
file = pd.read_csv("results.csv")
file

,Module_names,Signal_power,Logic_power,Static_Power(W),DC_power(W),Total_on_chip_power(W),Data_Path_Delay(ns),F7_Muxes,LUTs_as_Memory,LUTs_as_Logic,...,LUT1,LUT2,LUT3,LUT4,LUT5,LUT6,LUT7,Slice_LUTs2,LUTs_as_logic2,LUTs_difference
0,top_CLA8,0.167,0.065,0.088,0.232,4.895,9.882,0,0,13,...,0,1,2,3,3,6,0,15,15,2
1,top_CLA16,0.500,0.194,0.156,0.694,11.046,12.463,0,0,35,...,0,6,4,8,5,22,0,45,45,10
2,top_CLA32,1.749,0.505,0.485,2.254,23.213,18.806,0,0,96,...,0,26,9,14,15,59,0,123,123,27
3,top_FIFO,2.113,1.717,0.280,3.830,15.772,8.537,88,0,275,...,4,3,3,3,43,225,0,281,281,6
4,top_traffic_controller,0.103,0.117,0.073,0.220,1.079,6.740,0,0,39,...,5,4,5,15,15,8,0,52,52,13
5,top_BA_4x4_Multiplier,0.143,0.080,0.083,0.223,4.047,10.258,0,0,16,...,0,1,0,5,0,12,0,18,18,2
6,top_mealy,0.121,0.085,0.072,0.206,0.694,6.930,0,0,12,...,2,0,3,3,3,3,0,14,14,2
7,top_moore,0.107,0.067,0.071,0.174,0.463,5.660,0,0,13,...,2,0,2,6,3,3,0,16,16,3
8,top_priority_encoder0,0.041,0.015,0.072,0.056,0.987,7.899,0,0,4,...,0,0,1,1,1,2,0,5,5,1
9,top_priority_encoder1,0.045,0.015,0.071,0.060,0.471,7.906,0,0,4,...,0,0,0,2,1,2,0,5,5,1
